In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

In [132]:
# Functions

# reverse the data of argument
def reverseData(data):
    data = data[::-1]
    return data

def getRound(data, decimals = 3):
    return np.around(data, decimals)

## given two curve a, b, then return the inversection point of two curves
def FindTheBuyPoint(a, b, mode):
    if(mode == "rsi_1"):
        buyPoint = np.argwhere(np.diff(np.sign(a - 20)) > 0).reshape(-1) + 0
    else:
        buyPoint = np.argwhere(np.diff(np.sign(a - b)) > 0).reshape(-1) + 0
    return buyPoint

def FindTheSellPoint(a, b, mode):
    if(mode == "rsi_1"):
        sellPoint = np.argwhere(np.diff(np.sign(a - 80)) < 0).reshape(-1) + 0
    else:
        sellPoint = np.argwhere(np.diff(np.sign(a - b)) < 0).reshape(-1) + 0
    return sellPoint

def GetDataFromDB(ip, userName, psw, tableName, sql):
    try:
        db = pymysql.connect(ip, userName, psw, tableName)
    except:
        print ("Error: unable to connect to DB")
        
    #create a cursor
    cursor = db.cursor()
    results = []
    try:
        cursor.execute(sql)
        results = cursor.fetchall()
    except:
        print ("Error: unable to fetch data from DB")
    
    return results
    db.close()
    
def DeterminedStatus(indicatorResultFast, indicatorResultSlow, result):
    longShortIndex = [i for i in range(len(result)) if result[i] == 1 or result[i] == -1]
    for i in range(len(longShortIndex) - 1):
        begin = longShortIndex[i]
        end = longShortIndex[i + 1]
        #print (begin, end)
        #print (result[begin], result[end])
        # find the series between begin and end
        selectedIndicatorResultFast = indicatorResultFast[begin + 1:end]
        if len(selectedIndicatorResultFast) == 0:
            continue
        selectedIndicatorResultSlow = indicatorResultSlow[begin + 1:end]
        if len(selectedIndicatorResultSlow) == 0:
            continue
        # find the distance of every elements
        distanceSeries = [abs(x1 - x2) for (x1, x2) in zip(selectedIndicatorResultFast, selectedIndicatorResultSlow)]
        
        # normalize all distance to [0, 1]
        maxDistance = max(distanceSeries)
        #distanceSeries = [i / maxDistance for i in distanceSeries]
        distanceSeries = [i * 2 / maxDistance for i in distanceSeries]
        
        # find the index of largest distance
        #largestIndex = begin + 1 + distanceSeries.index(1)
        largestIndex = begin + 1 + distanceSeries.index(2)
        result[largestIndex] = 0
        firstNumber, lastNumber = result[begin], result[end]
        #print (begin + 1, largestIndex, largestIndex + 1, end - 1)
        '''
        for j in range(begin + 1, largestIndex):
            result[j] = firstNumber - distanceSeries[j - (begin + 1)]
        for j in range(largestIndex + 1, end):
            result[j] = lastNumber + distanceSeries[j - (begin + 1)]
        '''
        for j in range(begin + 1, end):
            if j != largestIndex:
                if firstNumber == 1:
                    result[j] = firstNumber - distanceSeries[j - (begin + 1)]
                else:
                    result[j] = firstNumber + distanceSeries[j - (begin + 1)]
        #print (begin, end)
        #print (distanceSeries)
        #print (result[begin:end + 1])
    return result

In [133]:
# indicators
def SMA(data, period):
    if (type(data) == pd.core.series.Series):
        return data.rolling(window = period).mean()
    else:
        print ("Error: Wrong input, SMA(pandas.core.series.Series, integer)")
        
def EMA(data, period):
    if (type(data) == pd.core.series.Series):
        return data.ewm(span = period, adjust = False).mean()
    else:
        print ("Error: Wrong input, EMA(pandas.core.series.Series, integer)")
        
def RSI(data, period):
    if (type(data) == pd.core.series.Series):
        delta = data.diff()
        up, down = delta.copy(), delta.copy()

        up[up < 0] = 0
        down[down > 0] = 0
        down = abs(down)
        
        Up = SMA(up, period)
        Up = Up.fillna(0)
        for i in range(period,len(Up)):
            Up[i] = getRound((Up[i - 1] * (period - 1) + up[i]) / period)

        Down = SMA(down, period)
        Down = Down.fillna(0)
        for i in range(period,len(Down)):
            Down[i] = getRound((Down[i - 1] * (period - 1) + down[i]) / period)


        rsi = 100 * (Up / (Up + Down))
        rsi = rsi.fillna(0)
        return getRound(rsi, 2)
    else:
        print ("Error: Wrong input, RSI(pandas.core.series.Series, integer)")
        
def MACD(data, period = []):
    if (type(data) == pd.core.frame.DataFrame and len(period) == 3):   
        di = (data['最高價(元)'] + data['最低價(元)'] + 2.0 * data['收盤價(元)']) / 4.0
        ema12 = SMA(di, period[0])
        ema12 = ema12.fillna(0)
        for i in range(period[0] + 1, len(ema12)):
            ema12[i] = (ema12[i - 1] * (period[0] - 1) + di[i] * 2.0) / (period[0] + 1)
    
        ema26 = SMA(di, period[1])
        ema26 = ema26.fillna(0)
        for i in range(period[1] + 1, len(ema26)):
            ema26[i] = (ema26[i - 1] * (period[1] - 1) + di[i] * 2.0) / (period[1] + 1)
    
        dif = ema12 - ema26

        dem = SMA(dif, period[2])
        dem = dem.fillna(0)
        for i in range(period[2] + 1, len(dem)):
            dem[i] = (dem[i - 1] * (period[2] - 1) + dif[i] * 2.0) / (period[2] + 1)
        return {'dif':dif, 'dem':dem}
    else:
        if(type(data) == pd.core.frame.DataFrame):
            print ("Error: Wrong input, MACD(pandas.core.frame.DataFrame, list of integer)")
        elif(len(period) == 3):
            print ("Error: number of content in list do not equal to 3")
            
def KD(data, result = {}):
    close = data['收盤價(元)'].copy()
    for i in range(0,9):
        close[i] = 0

    data['RSV'] = (( data['收盤價(元)'] - data['最低價(元)'].rolling(window = 9).min()) / (data['最高價(元)'].rolling(window = 9).max() - data['最低價(元)'].rolling(window = 9).min()))
    data['RSV'] = data['RSV'].fillna(0)
    if(not result):
        result = {
            'K9':[0],
            'D9' :[0]
        }
    #calculate everyday's KD
    for i in range(1, len(data.index)):
        K9_value = (1.0/3.0) * data['RSV'][i] + (2.0 / 3.0) * result['K9'][i - 1]
        result['K9'].append(getRound(K9_value, 5))
        D9_value = (2.0/3.0) * result['D9'][i - 1] + (1.0 / 3.0) * result['K9'][i]
        result['D9'].append(getRound(D9_value, 5))
        
    return result

In [134]:
# get data
etfPriceData = pd.read_csv(r"C:\Users\user\Desktop\TBrain_DataSet\taetfp.csv", encoding="big5")

In [139]:
# kd
KD_val = pd.DataFrame(KD(etfPriceData))
# kd's long points & short points
KD_long = FindTheBuyPoint(KD_val['K9'], KD_val['D9'], "kd")
KD_short = FindTheSellPoint(KD_val['K9'], KD_val['D9'], "kd")

# create a long/short points list
KD_result = [0 for i in range(len(KD_val))]

for i in KD_long:
    KD_result[i] = 1
for i in KD_short:
    KD_result[i] = -1
    
KD_result = DeterminedStatus(list(KD_val['K9']), list(KD_val['D9']), KD_result)

KD_result = pd.Series(KD_result).fillna(0).tolist()

In [142]:
# rsi
rsi5_result = pd.DataFrame(RSI(etfPriceData['收盤價(元)'], 5))
rsi5_result = rsi5_result.rename(columns={'收盤價(元)': 'rsi'})

rsi14_result = pd.DataFrame(RSI(etfPriceData['收盤價(元)'], 14))
rsi14_result = rsi14_result.rename(columns={'收盤價(元)': 'rsi'})

# rsi's long points & short points
rsi_long_with1curve = FindTheBuyPoint(rsi14_result['rsi'], [], "rsi_1")
rsi_short_with1curve = FindTheSellPoint(rsi14_result['rsi'], [], "rsi_1")

rsi_long_with2curve = FindTheBuyPoint(rsi5_result['rsi'], rsi14_result['rsi'], "rsi_2")
rsi_short_with2curve = FindTheSellPoint(rsi5_result['rsi'], rsi14_result['rsi'], "rsi_2")

# create a long/short points list
rsi1curve_result = [0 for i in range(len(rsi14_result))]
rsi2curve_result = [0 for i in range(len(rsi14_result))]

for i in rsi_long_with1curve:
    rsi1curve_result[i] = 1
for i in rsi_short_with1curve:
    rsi1curve_result[i] = -1
for i in range(len(rsi1curve_result)):
    if rsi1curve_result[i] == 0:
        rsi1curve_result[i] = -1.0 + 2.0 * (rsi14_result['rsi'][i] - 20.0) / (80.0 - 20.0)
rsi1curve_result = pd.Series(rsi1curve_result).fillna(0).tolist()

for i in rsi_long_with2curve:
    rsi2curve_result[i] = 1
for i in rsi_short_with2curve:
    rsi2curve_result[i] = -1
    
rsi2curve_result = DeterminedStatus(list(rsi5_result['rsi']), list(rsi14_result['rsi']), rsi2curve_result)

rsi2curve_result = pd.Series(rsi2curve_result).fillna(0).tolist()

In [7]:
# sma
sma_result_5 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 5))
sma_result_10 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 10))
sma_result_5 = sma_result_5.rename(columns={'收盤價(元)':'sma5'})
sma_result_10 = sma_result_10.rename(columns={'收盤價(元)':'sma10'})

sma_result_20 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 20))
sma_result_60 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 60))
sma_result_20 = sma_result_20.rename(columns={'收盤價(元)':'sma20'})
sma_result_60 = sma_result_60.rename(columns={'收盤價(元)':'sma60'})

sma_result_120 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 120))
sma_result_240 = pd.DataFrame(SMA(etfPriceData['收盤價(元)'], 240))
sma_result_120 = sma_result_120.rename(columns={'收盤價(元)':'sma120'})
sma_result_240 = sma_result_240.rename(columns={'收盤價(元)':'sma240'})

sma_result_5.fillna(value=0, inplace=True)
sma_result_10.fillna(value=0, inplace=True)
sma_result_20.fillna(value=0, inplace=True)
sma_result_60.fillna(value=0, inplace=True)
sma_result_120.fillna(value=0, inplace=True)
sma_result_240.fillna(value=0, inplace=True)

# sma's long points & short points
smas_long = FindTheBuyPoint(sma_result_5['sma5'], sma_result_10['sma10'], "sma")
smas_short = FindTheSellPoint(sma_result_5['sma5'], sma_result_10['sma10'], "sma")

smam_long = FindTheBuyPoint(sma_result_20['sma20'], sma_result_60['sma60'], "sma")
smam_short = FindTheSellPoint(sma_result_20['sma20'], sma_result_60['sma60'], "sma")

smal_long = FindTheBuyPoint(sma_result_120['sma120'], sma_result_240['sma240'], "sma")
smal_short = FindTheSellPoint(sma_result_120['sma120'], sma_result_240['sma240'], "sma")

# create a long/short points list
smas_result = [0 for i in range(len(sma_result_5))]
smam_result = [0 for i in range(len(sma_result_5))]
smal_result = [0 for i in range(len(sma_result_5))]

for i in smas_long:
    smas_result[i] = 1
for i in smas_short:
    smas_result[i] = -1
for i in range(len(smas_result)):
    if smas_result[i] == 0:
        if abs(sma_result_5['sma5'][i]) < abs(sma_result_10['sma10'][i]):
            smas_result[i] = sma_result_5['sma5'][i] / sma_result_10['sma10'][i]
        else:
            smas_result[i] = sma_result_10['sma10'][i] / sma_result_5['sma5'][i]
smas_result = pd.Series(smas_result).fillna(0).tolist()

for i in smam_long:
    smam_result[i] = 1
for i in smam_short:
    smam_result[i] = -1
for i in range(len(smam_result)):
    if smam_result[i] == 0:
        if abs(sma_result_20['sma20'][i]) < abs(sma_result_60['sma60'][i]):
            smam_result[i] = sma_result_20['sma20'][i] / sma_result_60['sma60'][i]
        else:
            smam_result[i] = sma_result_60['sma60'][i] / sma_result_20['sma20'][i]
smam_result = pd.Series(smam_result).fillna(0).tolist()

for i in smal_long:
    smal_result[i] = 1
for i in smal_short:
    smal_result[i] = -1
for i in range(len(smal_result)):
    if smal_result[i] == 0:
        if abs(sma_result_120['sma120'][i]) < abs(sma_result_240['sma240'][i]):
            smal_result[i] = sma_result_120['sma120'][i] / sma_result_240['sma240'][i]
        else:
            smal_result[i] = sma_result_240['sma240'][i] / sma_result_120['sma120'][i]
smal_result = pd.Series(smal_result).fillna(0).tolist()

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in double_scalars


In [143]:
# macd
MACD_val = pd.DataFrame(MACD(etfPriceData, [12,26,9]))
# kd's long points & short points
MACD_long = FindTheBuyPoint(MACD_val['dif'], MACD_val['dem'], "macd")
MACD_short = FindTheSellPoint(MACD_val['dif'], MACD_val['dem'], "macd")

# create a long/short points list
MACD_result = [0 for i in range(len(MACD_val))]

for i in MACD_long:
    MACD_result[i] = 1
for i in MACD_short:
    MACD_result[i] = -1

MACD_result = DeterminedStatus(list(MACD_val['dif']), list(MACD_val['dem']), MACD_result)
print (MACD_result)
print ([i for i in MACD_result if i > 1 or i < -1])
'''
for i in range(len(MACD_result)):
    if MACD_result[i] == 0:
        if abs(MACD_val['dif'][i]) < abs(MACD_val['dem'][i]):
            MACD_result[i] = MACD_val['dif'][i] / MACD_val['dem'][i]
        else:
            MACD_result[i] = MACD_val['dem'][i] / MACD_val['dif'][i]
'''
MACD_result = pd.Series(MACD_result).fillna(0).tolist()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -0.5986099601700128, -0.277020873216441, -0.02102385955599373, 0.18355094396570149, 0.34948978376438067, 0.4827709727467041, 0.5871795561608906, 0.6672518179294786, 0.7303276207774986, 0.7817504035522305, 0.8219261296992117, 0.8523146274746098, -1, 0, 0.5958644345928683, 0.27527275988912625, 0.018020432200816172, -0.18618445712421106, -0.34844299964629855, -0.47885336442484483, -0.5826026695653677, -0.6638338383757886, -0.7290215894416262, -0.7822700061862377, -0.8233511651115477, -0.857381130792916, -0.8856674018612963, -0.9080717194204114, -0.9270906352687788, -0.9423306193052972, -0.9535916317062563, -0.9624399222066147, -0.9691642391198434, -0.9739427956628612, -0.9756223557805547, -0.977778699343864, -0.9790938373674499, -0.980893181470651, -0.9824782577814959, -0.9854574541728757, -0.9878942467324952, -0.9904387282354028, -0.992157815592268, -0.9950205419069019, -0.9971681770171726, -0.9981687699781728, -0.9993271247077308, -0.9969868817598997